# Additional Usage with psycopg2

### Imports
```
import psycopg2
import age
```
### Connect to PostgreSQL(with AGE extention)

In [1]:
import psycopg2
import age

# Connect to PostgresDB with AGE
conn = psycopg2.connect(host="172.17.0.3", port="5432", dbname="postgres", user="postgres", password="agens")
cursor = conn.cursor()

# load AGE extention
cursor.execute("LOAD 'age';")
cursor.execute("SET search_path = ag_catalog, '$user', public;")

# Check if age_graph named 'testGraph' exists
cursor.execute("SELECT count(*) FROM ag_graph WHERE name=%s", ('testGraph',))

if cursor.fetchone()[0] == 0 :
# if age_graph named 'testGraph', create that
    cursor.execute("SELECT create_graph(%s);", ('testGraph',))


### Create & Query Vertices

In [2]:
# Create Vertices
age.execCypher(cursor, "testGraph", "CREATE (n:Person {name: 'Joe'})")
age.execCypher(cursor, "testGraph", "CREATE (n:Person {name: 'Smith'})")
age.execCypher(cursor, "testGraph", "CREATE (n:Person {name: %s})", ('Jack',))
age.execCypher(cursor, "testGraph", "CREATE (n:Person {name: 'Andy', title: 'Developer'})")
age.execCypher(cursor, "testGraph", "CREATE (n:Person {name: %s, title: %s})", ('Tom','Developer',))
conn.commit()

# Query Vertices
age.queryCypher(cursor, "testGraph", "MATCH (n:Person) RETURN n")

# parse each row in cursor
print("---[Parse & print vertices row by row]---")
for row in age.getRows(cursor):
    print(row.id, row.label, row.properties)

    
# Query Vertices with parameters
age.queryCypher(cursor, "testGraph", "MATCH (n:Person {name:%s}) RETURN n", ("Andy",))

# Parse full data in cursor
print("---[Parse all vertices in result set]---")
graph = age.buildGraph(cursor)

print("Count of queried vertices:", graph.size())

print("---[Print vertices in Graph object]---")
for row in graph:
    print(row.id, row.label, row.properties)
    

---[Parse & print vertices row by row]---
844424930132231 Person {'name': 'Joe'}
844424930132232 Person {'name': 'Smith'}
844424930132233 Person {'name': 'Jack'}
844424930132234 Person {'name': 'Andy', 'title': 'Developer'}
844424930132235 Person {'name': 'Tom', 'title': 'Developer'}
---[Parse all vertices in result set]---
Count of queried vertices: 1
---[Print vertices in Graph object]---
844424930132234 Person {'name': 'Andy', 'title': 'Developer'}


### Create & Query Edges

In [3]:
# Create Edges
age.execCypher(cursor, "testGraph", "MATCH (a:Person), (b:Person) WHERE a.name = 'Joe' AND b.name = 'Smith' CREATE (a)-[r:workWith {weight: 3}]->(b)")
age.execCypher(cursor, "testGraph", "MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith {weight: 1}]->(b)")
age.execCypher(cursor, "testGraph", "MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'Andy'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
age.execCypher(cursor, "testGraph", "MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Jack'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
conn.commit()

print("-- [Query paths] --------")
age.queryCypher(cursor, "testGraph", "MATCH p=()-[:workWith]-() RETURN p")

print("---[Parse & print paths row by row]---")
for path in age.getRows(cursor):
    print(path.start["name"], '-[',path.rel.label, path.rel.properties, ']-', path.end["name"])


-- [Query paths] --------
---[Parse & print paths row by row]---
Joe -[ workWith {'weight': '3'} ]- Smith
Smith -[ workWith {'weight': '3'} ]- Joe
Andy -[ workWith {'weight': '1'} ]- Tom
Tom -[ workWith {'weight': '1'} ]- Andy
Jack -[ workWith {'weight': '5'} ]- Andy
Andy -[ workWith {'weight': '5'} ]- Jack
Joe -[ workWith {'weight': '5'} ]- Jack
Jack -[ workWith {'weight': '5'} ]- Joe


## Query Scalar or properties value

In [4]:
# Query scalar value
print("-- Query scalar value --------------------")
age.queryCypher(cursor, "testGraph", "MATCH (n:Person) RETURN id(n)")
value = age.getSingle(cursor)
print(value)
    
# Query properties 
print("-- Query properties --------------------")
age.queryCypher(cursor, "testGraph", "MATCH (n:Person) RETURN properties(n)")
value = age.getSingle(cursor)
print(value)
    
# Query properties value
print("-- Query properties value --------------------")
age.queryCypher(cursor, "testGraph", "MATCH (n:Person {name: 'Andy'}) RETURN n.title")
value = age.getSingle(cursor)
print(value)
    
print("-- Query path count --------")
age.queryCypher(cursor, "testGraph", "MATCH p=(:Person {name: 'Andy'})-[:workWith]-() RETURN count(p)")
graph = age.buildGraph(cursor)
print(graph[0])

-- Query scalar value --------------------
844424930132231
-- Query properties --------------------
{'name': 'Joe'}
-- Query properties value --------------------
Developer
-- Query path count --------
2


## Close connection

In [5]:
# Clear test data
age.queryCypher(cursor, "testGraph", "MATCH (n:Person) DETACH DELETE n RETURN *")

conn.commit()

conn.close()